# anchors

In [1]:
from lib.layutils.generate_anchors import *
# 围绕中心点，生成9个初始化的anchors用于后期与实际目标进行比较，得到其误差
a = generate_anchors()
print(a)

[ 0  0 15 15]
[[-3.5  2.  18.5 13. ]
 [ 0.   0.  15.  15. ]
 [ 2.5 -3.  12.5 18. ]]
[[ -84.  -40.   99.   55.]
 [-176.  -88.  191.  103.]
 [-360. -184.  375.  199.]
 [ -56.  -56.   71.   71.]
 [-120. -120.  135.  135.]
 [-248. -248.  263.  263.]
 [ -36.  -80.   51.   95.]
 [ -80. -168.   95.  183.]
 [-168. -344.  183.  359.]]


# data load data

In [3]:
from lib.datasets.factory import get_imdb
from lib.datasets.pascal_voc import pascal_voc
d = pascal_voc('trainval', '2007')
res = d.roidb
# print(res)
# imdb_names = 'voc_2007_trainval'
# imdb = get_imdb(imdb_names)

<bound method imdb.default_roidb of <lib.datasets.pascal_voc.pascal_voc object at 0x0000022CB38CDAC8>>
voc_2007_trainval gt roidb loaded from D:\PROJECT_TW\git\data\voc_2007_trainval_gt_roidb.pkl


# NMS

# ROI

    https://deepsense.ai/region-of-interest-pooling-in-tensorflow-example/
    https://www.cnblogs.com/king-lps/p/9031568.html

# Cython

    https://www.cnblogs.com/freeweb/p/6548208.html
    https://deepsense.ai/region-of-interest-pooling-in-tensorflow-example/
    https://www.cnblogs.com/king-lps/p/9031568.html